![imagenes](logo.png)

# Unión de dataframes

Otra tarea común de transformación de datos es "unir" o "fusionar" dos conjuntos de datos diferentes. Por ejemplo, en el dataframe de vuelos, la variable ``carrier`` dá el código de la compañía aérea para los diferentes vuelos. Si bien los nombres de las aerolíneas correspondientes a "UA" y "AA" pueden ser algo fáciles de adivinar (United y American Airlines), ¿qué aerolíneas tienen los códigos "VX", "HA" y "B6"? Esta información se proporciona en un dataframe separado llamado ``airlines``.

In [ ]:
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
import os

os.chdir("C:/Users/Usuario/Documents/scidata/24_cdd_py/practicas/aerolineas")

In [ ]:
airlines = pd.read_csv("airlines.csv")
flights = pd.read_csv("flights.csv")
airports = pd.read_csv("airports.csv")
weather = pd.read_csv("weather.csv")

In [ ]:
airlines >> head()

In [ ]:
flights >> head()

Nota que los valores de la variable ``carrier`` en el dataframe ``flights`` coinciden con los valores de la variable ``carrier`` en el dataframe ``airlines``. En este caso, podemos usar la variable ``carrier`` como variable *clave* para hacer coincidir las filas de los dos dataframes. Las variables clave casi siempre son variables de identificación que identifican de manera única las unidades de observación, como vimos anteriormente. Esto garantiza que las filas de ambos marcos de datos coincidan adecuadamente durante la unión. Hadley y Garrett (Grolemund y Wickham 2017) crearon el diagrama que se muestra en la figura siguiente para ayudarnos a comprender cómo los diferentes dataframes de la carpeta ``nycflights13`` están vinculados por varias variables clave:

![imagenes](im010.png)

## Coincidencia de nombres de variables "clave"

Tanto en el dataframe ``flights`` como en ``airlines``, la variable clave por la que queremos unir/fusionar/coincidir las filas tiene el mismo nombre: ``carrier``. Usemos la función ``inner_join()`` para unir los dos dataframes, donde las filas coincidirán con el variable ``carrier``, y luego comparemos los dataframes resultantes:

In [ ]:
# flights_joined =
flights >> inner_join(_,airlines, on = "carrier")

In [ ]:
flights.columns

In [ ]:
flights_joined

Observa que los dataframes ``flights`` y ``flights_joined`` son idénticos, excepto que ``flights_joined`` tiene una variable adicional. Los valores de ``name`` corresponden a los nombres de las compañías aéreas como se indica en el datafrema ``airlines``.

En la figura siguiente se muestra una representación visual de ``inner_join()``. Hay otros tipos de uniones disponibles (como ``left_join()``, ``right_join()``, ``outside_join()`` y ``anti_join()``), pero ``inner_join()`` y ``left_join()`` suelen ser los mas importantes.

![imagenes](im011.png)

## Diferentes nombres de variables "clave"

En cambio, digamos que estás interesado en los destinos de todos los vuelos nacionales que salen de la ciudad de Nueva York en 2013 y te haces preguntas como: ¿En qué ciudades se encuentran estos aeropuertos?, ¿Es "ORD" Orlando? o ¿Dónde está "FLL"?.

El dataframe ``airports`` contiene los códigos de aeropuerto para cada aeropuerto:

In [ ]:
airports.columns

In [ ]:
airports >> head()

Sin embargo, si observas los dataframes ``airports`` y ``flights``, encontrará que los códigos de aeropuerto están en variables que tienen nombres diferentes. En ``airports`` el código de aeropuerto está en ``faa``, mientras que en ``fligths`` los códigos de aeropuerto están en ``origin`` y ``dest``. Este hecho se destaca aún más en la representación visual de las relaciones entre estos marcos de datos de la figura mas arriba.

Para unir estos dos dataframes por código de aeropuerto, nuestra operación ``inner_join()`` usará el argumento ``by = c("dest" = "faa")`` con sintaxis de código modificada que nos permite unir dos dataframes donde la variable clave tiene un nombre diferente:

In [ ]:
# flights_with_airport_names =

flights >> inner_join(_, airports, on = {'dest': 'faa'}) 

# flights_with_airport_names

Construyamos la cadena de operadores de tubería ``>>`` que calcula la cantidad de vuelos desde Nueva York a cada destino, pero también incluye información sobre cada aeropuerto de destino:

In [ ]:
(
flights >>
  group_by(_.dest) >>
  summarize(num_flights = n(_)) >>
  arrange(-_.num_flights)  >>
  inner_join(_,airports, on = {'dest': 'faa'}) >>
  rename(airport_name = _.name) >>
  select(-_.faa)
)


Por si no lo sabías, "ORD" es el código de aeropuerto del aeropuerto O'Hare de Chicago y "FLL" es el aeropuerto principal de Fort Lauderdale, Florida, que se puede ver en la variable ``airport_name``.

# Múltiples variables "clave"

En cambio, digamos que queremos unir dos dataframes mediante múltiples variables clave. Por ejemplo, en la figura de relaciones vemos que para unir los dataframes ``fligths`` y ``weather``, necesitamos más de una variable clave: ``year``, ``month``, ``day``, ``hour`` y ``origin``. Esto se debe a que la combinación de estas 5 variables actúa para identificar de manera única cada unidad de observación en el dataframe ``weather``: registros meteorológicos por hora en cada uno de los 3 aeropuertos de Nueva York.

Logramos esto especificando una lista de variables clave para unir las tablas.

In [ ]:
flights >> head()

In [ ]:
weather 

In [ ]:
flights >> inner_join(_,weather, on = ["year", "month", "day", "hour", "origin"])

#flights_weather_joined

---
---
**Ejercicios.**

**8.1.** Mirando la figura de relaciones, al unir ``fligths`` y ``weather`` (o, en otras palabras, hacer coincidir los valores del clima por hora con cada vuelo), ¿por qué necesitamos unir todo el año, mes, día, hora y origen, y no solo hora?

**8.2.** ¿Qué le sorprende de los 10 mejores destinos desde NYC en 2013?

---
---

## Formas normales

Los dataframes incluidos en la carpeta ``nycflights13`` tienen un formato que minimiza la redundancia de datos. Por ejemplo, ``flights`` solo guarda el código de la aerolínea de la compañía aérea; no incluye el nombre real de la aerolínea. 

Los nombres de las compañías aéreas se incluyen en la variable ``name`` de ``airlines``. Para incluir el nombre de la compañía aérea en los vuelos, podríamos unir estos dos dataframes de la siguiente manera:

In [ ]:
flights >> inner_join(_, airlines, on = "carrier")

Somos capaces de realizar esta unión porque cada uno de los dataframes tiene claves en común para relacionarse entre sí: la variable ``carrier`` tanto en ``flights`` como en ``airlines``. Las variables clave en las que basamos nuestras uniones suelen ser variables de identificación, como mencionamos anteriormente.

Esta es una propiedad importante de lo que se conoce como **formas normales de datos**. El proceso de descomposición de dataframes en tablas menos redundantes sin perder información se denomina normalización. Puedes ver mas información en nuestro [repositorio de Arquitectura y Administración de bases de datos de 2024](https://github.com/scidatmath2020/AABD-2024/blob/main/C05.%20Normalizaci%C3%B3n%20de%20una%20base%20de%20datos.ipynb)

Tanto ``siuba`` como **SQL** y ``dplyr`` de **R** usan formas normales. Dado que comparten tales puntos en común, una vez que aprendes cualquiera de estas herramientas, puedes aprender las otras muy fácilmente.

---
---

**Ejercicios**

**8.3.** ¿Cuáles son algunas de las ventajas de los datos en formas normales? ¿Cuáles son algunas desventajas?

---
---